# 1. Introduction

## 1.1 Problem Statement


We are small investors and plan to open a restaurant in Ho Chi Minh City. In order to optimize the selection of location and scale of our restaurant, we shall consider the below criteria:
- Competitiveness: The neighborhood is not so competitive given (i) there are not too many restaurants in the neighborhood or (ii) the restaurants in the neighborhood are not highly rated in Foursquare.

- Neighborhood's population: The bigger population the better as we have a larger pool of potential customers. We do not want to operate our restaurant in suburban areas where there are not too many competitors but a small pool of potential customers as well.

- The restaurant tier (cheap, medium or expensive restaurant): This depends on the neighborhood's preference. We cannot open an highly expensive restaurant in a low-middle neighborhood and vice versa.

## 1.2. Data

Base on the criteria pre-defined above, we need the following data to perform the task:

- List of suggested venues from Foursquare API containing (i) venue's coordinates, (ii) venue's rating, and (iii) venue's tier.

- Population at district level.


Data source:

- Latlong.net to get coordinates of HCMC and its districts.

- Foursquare: Use regular calls to get suggested venues and their location. Use premimum calls to get venue's rating and tier.

- Modoho.com.vn to get population at district level.

## 1.3. Methodology

We will use a combination of clustering models and map visualization to do the analyses.

## 2. Data Collection

## 2.1. First step, import all neccesary libraries

In [1]:
import folium
import json
# import matplotlib.cm as cm
# import matplotlib.colors as colors
# import matlibplot.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns
import requests
from sklearn.cluster import KMeans

pd.set_option('display.max_rows', None)

## 2.2. Get list of recommended venues from Foursquare

### 2.2.1. Get coordinates for centroids of our candidate neighborhoods


In order to get as many recommended venues as possible, we will do the following steps:


a. From the central point of the city, we look for all venues within a radius of 12 km. We do not want to open a restaurant too far way from city center becuase it is just personal preference.

b. Break down the exploring area to smaller candidate neighborhoods with a radius of 300 meters.

c. Iterate through all centroids of those candidate neighborhoods, using its coordinates to request Foursquare exploration API calls to get all recommended venues within the neighborhoods.

*Note: The section below is copied from https://cocl.us/coursera_capstone_notebook project. Credits to the unknown author.

Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Alexanderplatz. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [7]:
hcm_center = (10.762622, 106.660172)

In [54]:
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=48, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=48, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

Convert longitudes and latitudes to Cartesian coordinates to calculate distances.

In [55]:
hcm_center_x, hcm_center_y = lonlat_to_xy(hcm_center[1], hcm_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = hcm_center_x - 6000
x_step = 600
y_min = hcm_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []

for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(hcm_center_x, hcm_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

Visualize the city center and candidate neighborhoods.

In [82]:
hcm_map = folium.Map(location=hcm_center, tiles='CartoDB dark_matter', zoom_start=13)

folium.Marker(hcm_center, popup='HCM').add_to(hcm_map)

for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(hcm_map) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(hcm_map)
    #folium.Marker([lat, lon]).add_to(hcm_map)

hcm_map

### 2.2.2. Get recommended venues within all candidate neighborhoods

Below are all functions necessary to requests Foursquare API and manipulate the results.

In [52]:
def get_exploring_urls(client_id, client_secret, version, latitidues, longitudes, category, radius=600, limit=100, time='any', day='any'):
    urls = [f'https://api.foursquare.com/v2/venues/explore?&client_id={client_id}&client_secret={client_secret}&v={version}&ll={lat},{long}&categoryId={category}&radius={radius}&limit={limit}&time={time}&day={day}' for lat, long in zip(latitudes, longitudes)]

    return urls


def get_venue_details_urls(client_id, client_secret, version, venue_ids):
    urls = [f'https://api.foursquare.com/v2/venues/{venue_id}?client_id={client_id}&client_secret={client_secret}&v={20200101}' for venue_id in venue_ids]

    return urls


def get_venue_tips_urls(client_id, client_secret, version, venue_ids, limit=500):
    urls = [f'https://api.foursquare.com/v2/venues/{venue_id}/tips?client_id={client_id}client_secret={client_secret}&v={version}&limit={limit}' for venue_id in venue_ids]

    return urls


def get_venue_menu_urls(client_id, client_secret, version, venue_ids):
    urls = [f'https://api.foursquare.com/v2/venues/venue_id/menu?client_id={client_id}&client_secret={client_secret}&v={version}' for venue_id in venue_ids]

    return urls


def request_api(urls):
    results = [requests.get(url).json() for url in urls]

    return results


def check_errors(api_results):
    errors = []
    for idx, result in enumerate(api_results):
        if result['meta']['code'] != 200:
            errors[version].append((idx, result))
    
    return errors


def remedy_errors(api_results, urls, errors, tries=5):
    while len(errors) > 0 and tries > 0:
        for error in errors:
            idx = error[0]
            url = urls[idx]
            
            new_result = requests.get(url).json()
            api_results[idx] = new_result
            
        errors = check_errors(api_results)
        tries -= 1
    
    return api_results


def get_venues(api_results):
    venues = []
    for result in api_results:
        if result['meta']['code'] == 200:
            for item in result['response']['groups'][0]['items']:
                venues.append(item['venue'])
    
    return venues


def get_distinct_venues(venues):
    venue_ids = []
    distinct_venues = []
    for venue in venues:
        venue_id = venue['id']
        if venue_id not in venue_ids:
            venue_ids.append(venue_id)
            distinct_venues.append(venue)
    
    return distinct_venues


def get_venue_coordinates(venues):
    coordinates = [(venue['location']['lat'], venue['location']['lng']) for venue in venues]

    return coordinates


def get_venues_tier(venues_details):
    venue_tier = []
    for venue in venues_details:
        try:
            tier = venue['response']['venue']['price']['tier']
        except:
            tier = None

        venue_tier.append(tier)

    return venue_tier


def get_venues_likes(venues_details):
    venue_likes = []
    for venue in venues_details:
        try:
            likes = venue['response']['venue']['likes']['count']
        except:
            likes = None
        
        venue_likes.append(likes)
    
    return venue_likes


def get_venues_rating(venues_details):
    venue_rating = []
    for venue in venues_details:
        try:
            rating = venue['response']['venue']['rating']
        except:
            rating = None
        
        venue_rating.append(rating)
    
    return venue_rating

Below are client ID and client secret used for API request. Hidden for privacy purpose.

In [71]:
client_id = 'VBT1KEJNYQMDORW3N55MCKB3S35RZJVEG3I42IN3SSCLMQZO' # your Foursquare ID
client_secret = 'ZW3MU1JZLNMKMIS4NBS1X4VPSMMGXRIW1LLXL5KC0AGEZMXY' # your Foursquare Secret
version = '20200101' # Foursquare API version

Below are codes to requests Foursquare API. No need to run because the results are saved in Google drive and will beloaded from there.

In [66]:
# food_category = '4d4b7105d754a06374d81259'

# urls_food = get_exploring_urls(client_id, client_secret, version, latitudes, longitudes, food_category)
# results_food = request_api(urls_food)
# errors_food = check_errors(results_food) # Sometimes the API requests fails, we need to check those errors and run again
# results_food = remedy_errors(results_food, urls_food, errors_food, tries=10) # Iteratively request results for those detected erroneous results
# print('Errors count:', len(check_errors(results_food)))

In [251]:
# venues_food = get_distinct_venues(get_venues(results_food))

# with open('results_food.json', 'w') as fp:
#     json.dump(results_food, fp)

# with open('venues_food.json', 'w') as fp:
#     json.dump(venues_food, fp)

In [ ]:
# nightlife_category = '4d4b7105d754a06376d81259'

# urls_nightlife = get_exploring_urls(client_id, client_secret, version, latitudes, longitudes, nightlife_category)
# results_nightlife = request_api(urls_nightlife)
# errors_nightlife = check_errors(results_nightlife)  # Sometimes the API requests fails, we need to check those errors and run again
# results_nightlife = remedy_errors(results_nightlife, urls_nightlife, errors_nightlife, tries=10) # Iteratively request results for those detected erroneous results
# print('Errors count:', len(check_errors(results_nightlife)))

In [252]:
# venues_nightlife = get_distinct_venues(get_venues(results_nightlife))

# with open('results_nightlife.json', 'w') as fp:
#     json.dump(results_nightlife, fp)

# with open('venues_nightlife.json', 'w') as fp:
#     json.dump(venues_nightlife, fp)

In [79]:
# # Getting Venues Details is a premium call and has a limitation of 500 calls a day. Thus, the codes below are use repeatedly in 5 days to get all details for more than 2,000 venues

# start_idx = len(venues_details_food)
# end_idx = start_idx + 500

# temp_venues_id_food = [venue['id'] for venue in venues_food[start_idx:end_idx]]
# temp_venues_details_urls_food = get_venue_details_urls(client_id, client_secret, version, temp_venues_id_food)
# temp_venues_details_food = request_api(temp_venues_details_urls_food)
# venues_details_food.extend(temp_venues_details_food)

# with open('venues_details_food.json', 'w') as fp:
#     json.dump(venues_details_food, fp)

In [78]:
# # Check for failed requests and make API call again

# for idx, venue in enumerate(venues_details_food):
#     if venue['meta']['code'] != 200:
#         venue_id = venues_food[idx]['id']
#         temp_url = get_venue_details_urls(client_id, client_secret, version, [venue_id])
#         temp_api_result = request_api(temp_url[0])
#         venue = temp_api_result[0]

In [130]:
# Load venues json files

with open('results_food.json', 'r') as f:
    results_food = json.load(f)

with open('venues_food.json', 'r') as f:
    venues_food = json.load(f)

with open('venues_details_food.json', 'r') as f:
    venues_details_food = json.load(f)

OSError: [Errno 22] Invalid argument: 'https://drive.google.com/file/d/1IinAcgRaZ7bcRnCzac9xdao_vpebI7xT/view?usp=sharing'

Visualize all venues on the city map

In [11]:
from folium.plugins import MarkerCluster

venues_coor_food = get_venue_coordinates(venues_food)
coordinates = venues_coor_food.copy()

hcm_map = folium.Map(location=hcm_center, tiles='CartoDB dark_matter', zoom_start=12)

marker_cluster = MarkerCluster().add_to(hcm_map)

for lat_long in coordinates:
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(hcm_map) 
    # folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(hcm_map)
    folium.Marker(lat_long).add_to(marker_cluster)

hcm_map

## 2.3. Data Analytics and Visualization

### 2.3.1. Data pre-processing

Firstly, we get the following information from Venue Details results:

a. Venue tier: ranging from 1 (cheap) to 4 (highly expensive).

b. Venue likes.

c. Venue rating.

In [80]:
venues_tier = get_venues_tier(venues_details_food)
venues_likes = get_venues_likes(venues_details_food)
venues_rating = get_venues_rating(venues_details_food)

Create a pandas dataframe containing the following information:

1. latitude and longitude.

2. x and y (converted from longitude and latitude, used for modeling).

3. tier.

4. likes.

5. rating.

In [82]:
length = len(venues_details_food)

venues_id_food = [venue['id'] for venue in venues_food]

venue_coords_food = get_venue_coordinates(venues_food)
transposed_venue_coords_food = np.array(venue_coords_food).transpose()
lats = transposed_venue_coords_food[0]
longs = transposed_venue_coords_food[1]
x, y = lonlat_to_xy(longs[:length], lats[:length])

venues_df = pd.DataFrame({
    'venue_id': venues_id_food[:length],
    'latitude': lats[:length],
    'longitude': longs[:length],
    'x': x,
    'y': y,
    'tier': venues_tier,
    'likes': venues_likes,
    'rating': venues_rating
})
venues_df.set_index('venue_id', inplace=True)
venues_df.describe()

,latitude,longitude,x,y,tier,likes,rating
count,1467.000000,1467.000000,1467.000000,1.467000e+03,925.00000,1463.000000,843.000000
mean,10.762854,106.677227,683396.101166,1.190260e+06,1.64000,14.300752,6.902372
std,0.015993,0.026123,2854.876000,1.772841e+03,0.63033,37.741721,0.776094
min,10.709257,106.604434,675429.144665,1.184310e+06,1.00000,0.000000,5.200000
25%,10.753930,106.662386,681773.090651,1.189271e+06,1.00000,1.000000,6.300000
50%,10.767480,106.685888,684337.960676,1.190777e+06,2.00000,6.000000,6.900000
75%,10.774776,106.697893,685649.760414,1.191589e+06,2.00000,12.000000,7.500000
max,10.786842,106.716021,687633.758432,1.192894e+06,4.00000,659.000000,9.400000


Remove latitude and longitude from features used for models. Drop all NA values

In [83]:
X = venues_df[['x', 'y', 'tier', 'likes', 'rating']]
X.dropna(inplace=True)
X.describe()

,x,y,tier,likes,rating
count,466.000000,4.660000e+02,466.000000,466.000000,466.000000
mean,684517.281299,1.190131e+06,1.697425,22.890558,6.832833
std,1754.274193,1.311932e+03,0.636416,53.432490,0.754125
min,676327.034915,1.186546e+06,1.000000,2.000000,5.200000
25%,683799.056991,1.189698e+06,1.000000,6.000000,6.200000
50%,684855.222722,1.190611e+06,2.000000,10.000000,6.800000
75%,685803.918209,1.191041e+06,2.000000,20.000000,7.300000
max,687285.104009,1.192051e+06,4.000000,659.000000,8.700000


Standardize features using for models

In [84]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)
scaled_X

array([[ 0.32789607, -2.71031006,  0.47594683, -0.39139124, -2.16752822],
       [ 0.58157009, -2.71217292, -1.09704055, -0.33518529, -0.70731667],
       [-0.92954789, -2.59638726,  0.47594683,  0.0769917 ,  0.08916235],
       ...,
       [ 1.014346  ,  1.40571149,  2.0489342 ,  3.99267315,  1.6821204 ],
       [ 0.75118504,  1.3241999 ,  0.47594683,  6.42826448,  0.48740186],
       [ 1.06489488,  1.29996464,  0.47594683,  0.43296274,  0.88564138]])

### 2.3.2. DBSCAN model

In [85]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.3, min_samples=4).fit(scaled_X)
X['db_labels'] = db.labels_
X.groupby('db_labels')[['x', 'y', 'tier', 'likes', 'rating']].describe().stack()

x             y        tier       likes  \
db_labels                                                              
-1        count     378.000000  3.780000e+02  378.000000  378.000000   
          mean   684386.539896  1.189994e+06    1.685185   26.208995   
          std      1895.680058  1.379117e+03    0.674541   58.803149   
          min    676327.034915  1.186546e+06    1.000000    2.000000   
          25%    683311.093137  1.189472e+06    1.000000    7.000000   
          50%    684755.237261  1.190429e+06    2.000000   11.000000   
          75%    685876.587583  1.191013e+06    2.000000   22.000000   
          max    687285.104009  1.192051e+06    4.000000  659.000000   
 0        count       4.000000  4.000000e+00    4.000000    4.000000   
          mean   686057.911358  1.187789e+06    2.000000    6.750000   
          std       216.574185  4.077445e+01    0.000000    1.500000   
          min    685755.160788  1.187751e+06    2.000000    5.000000   
          25%    685975.799330  1.187759e+06    2.000000    5.750000   
          50%    686123.372869  1.187783e+06    2.000000    7.000000   
          75%    686205.484897  1.187813e+06    2.000000    8.000000   
          max    686229.738906  1.187840e+06    2.000000    8.000000   
 1        count      12.000000  1.200000e+01   12.000000   12.000000   
          mean   684483.340497  1.190380e+06    2.000000    7.916667   
          std       209.928334  4.500364e+02    0.000000    5.775471   
          min    684017.344146  1.189716e+06    2.000000    3.000000   
          25%    684409.481643  1.190016e+06    2.000000    4.750000   
          50%    684516.541640  1.190334e+06    2.000000    5.500000   
          75%    684651.535969  1.190779e+06    2.000000    8.750000   
          max    684716.928936  1.190991e+06    2.000000   22.000000   
 2        count       4.000000  4.000000e+00    4.000000    4.000000   
          mean   684055.918940  1.190711e+06    1.000000    7.250000   
          std       113.207422  1.350306e+02    0.000000    3.095696   
          min    683930.972841  1.190591e+06    1.000000    3.000000   
          25%    683976.659990  1.190614e+06    1.000000    6.000000   
          50%    684060.657271  1.190683e+06    1.000000    8.000000   
          75%    684139.916220  1.190780e+06    1.000000    9.250000   
          max    684171.388375  1.190888e+06    1.000000   10.000000   
 3        count      33.000000  3.300000e+01   33.000000   33.000000   
          mean   685179.213265  1.191027e+06    2.000000    9.363636   
          std       506.076854  2.297968e+02    0.000000    3.967195   
          min    684307.003192  1.190590e+06    2.000000    4.000000   
          25%    684900.611399  1.190900e+06    2.000000    7.000000   
          50%    685165.993003  1.191025e+06    2.000000    8.000000   
          75%    685519.091370  1.191234e+06    2.000000   10.000000   
          max    686351.704082  1.191324e+06    2.000000   20.000000   
 4        count       8.000000  8.000000e+00    8.000000    8.000000   
          mean   685074.279378  1.190555e+06    2.000000   13.750000   
          std       250.852459  1.609170e+02    0.000000    7.535630   
          min    684830.731965  1.190304e+06    2.000000    5.000000   
          25%    684892.840418  1.190484e+06    2.000000    8.500000   
          50%    684953.697385  1.190624e+06    2.000000   12.500000   
          75%    685308.276528  1.190646e+06    2.000000   16.750000   
          max    685443.262608  1.190735e+06    2.000000   27.000000   
 5        count       4.000000  4.000000e+00    4.000000    4.000000   
          mean   683785.167817  1.190881e+06    1.000000    7.250000   
          std       262.164124  1.470654e+02    0.000000    2.362908   
          min    683458.259516  1.190736e+06    1.000000    4.000000   
          25%    683703.898783  1.190767e+06    1.000000    6.250000   
          50%    683791.148283  1.190874e+06    1.000000    8.000000 

### 2.3.3. K-Means Model

In [86]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, random_state=0).fit(scaled_X)
X['kmeans_labels'] = kmeans.labels_
X.groupby('kmeans_labels')[['x', 'y', 'tier', 'likes', 'rating']].describe().stack()

x             y        tier       likes  \
kmeans_labels                                                              
0             count     103.000000  1.030000e+02  103.000000  103.000000   
              mean   684683.000822  1.190598e+06    1.000000   16.844660   
              std      1012.685214  5.820605e+02    0.000000   22.197755   
              min    682072.884849  1.189172e+06    1.000000    2.000000   
              25%    683992.123839  1.190186e+06    1.000000    6.000000   
              50%    684568.259221  1.190754e+06    1.000000    9.000000   
              75%    685297.243904  1.191077e+06    1.000000   18.000000   
              max    686710.785568  1.191311e+06    1.000000  143.000000   
1             count      58.000000  5.800000e+01   58.000000   58.000000   
              mean   686164.774244  1.187156e+06    1.672414   10.413793   
              std       817.128819  5.739960e+02    0.603729    7.862937   
              min    682888.350026  1.186546e+06    1.000000    2.000000   
              25%    685882.054678  1.186633e+06    1.000000    5.250000   
              50%    686301.749116  1.186914e+06    2.000000    7.500000   
              75%    686753.263359  1.187727e+06    2.000000   12.750000   
              max    687285.104009  1.188554e+06    3.000000   39.000000   
2             count     121.000000  1.210000e+02  121.000000  121.000000   
              mean   684970.809193  1.190797e+06    1.933884   26.570248   
              std       878.218379  5.515437e+02    0.460716   30.062941   
              min    682565.014450  1.188976e+06    1.000000    4.000000   
              25%    684380.311458  1.190534e+06    2.000000    9.000000   
              50%    685045.991913  1.190935e+06    2.000000   14.000000   
              75%    685639.853385  1.191219e+06    2.000000   35.000000   
              max    686694.255242  1.192051e+06    3.000000  183.000000   
3             count     104.000000  1.040000e+02  104.000000  104.000000   
              mean   685039.361686  1.190657e+06    2.250000   12.182692   
              std      1089.197173  5.149973e+02    0.477656   11.888450   
              min    680821.142539  1.189355e+06    2.000000    2.000000   
              25%    684435.559270  1.190317e+06    2.000000    5.000000   
              50%    684907.632597  1.190752e+06    2.000000    8.000000   
              75%    686136.865640  1.191134e+06    2.000000   15.000000   
              max    686823.658489  1.191316e+06    4.000000   84.000000   
4             count       7.000000  7.000000e+00    7.000000    7.000000   
              mean   685683.660918  1.191576e+06    2.000000  401.285714   
              std       550.587542  4.613860e+02    0.577350  132.238061   
              min    684856.498765  1.190777e+06    1.000000  236.000000   
              25%    685347.465235  1.191318e+06    2.000000  352.000000   
              50%    685833.651134  1.191729e+06    2.000000  368.000000   
              75%    686052.867028  1.191920e+06    2.000000  421.000000   
              max    686294.812004  1.192048e+06    3.000000  659.000000   
5             count      73.000000  7.300000e+01   73.000000   73.000000   
              mean   681367.122469  1.189843e+06    1.493151   14.205479   
              std      1314.296376  7.740047e+02    0.580309   12.763837   
              min    676327.034915  1.187922e+06    1.000000    3.000000   
              25%    680738.992247  1.189294e+06    1.000000    7.000000   
              50%    681481.132645  1.189718e+06    1.000000   10.000000   
              75%    682351.207069  1.190372e+06    2.000000   18.000000   
              max    683361.037581  1.191253e+06    3.000000   88.000000   

                         rating  
kmeans_labels                    
0             count  103.000000  
              mean     6.479612  
              std      0.524004  
              min      5.200000  
              25%      6.100

In [87]:
joined_X = X.join(venues_df[['latitude', 'longitude']], how='left')

### 2.3.4. Districts' GeoJSON and demographics

In [88]:
districts_demo = pd.read_csv(r'districts_demographics.csv')
districts_demo['pop_density'] = districts_demo['population']/districts_demo['area']

exl_districts = ['district 9', 'district 12', 'go vap district', 'thu duc district', 'hoc mon district', 'can gio district', 'nha be district', 'cu chi district']
districts_demo = districts_demo[~districts_demo['district'].isin(exl_districts)]
districts_demo.reset_index(drop=True, inplace=True)

districts_demo

,district,population,no. of wards / communes,area,pop_density
0,district 1,193632,10,7.73,25049.417853
1,district 2,147168,11,49.74,2958.745476
2,district 3,196333,14,4.92,39905.081301
3,district 4,186727,15,4.18,44671.531100
4,district 5,178615,15,4.27,41830.210773
5,district 6,258945,14,7.19,36014.603616
6,district 7,310178,10,35.69,8690.893808
7,district 8,431969,16,19.18,22521.845673
8,district 10,238558,15,5.72,41705.944056
9,district 11,230596,16,5.14,44863.035019


In [3]:
hcm_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=1973756&params=0'
hcm_geo = requests.get(hcm_geo_url).json()

In [46]:
d1_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=2587287&params=0'
d2_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=3799817&params=0'
d3_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=3819816&params=0'
d4_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=2778323&params=0'
d5_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=3820432&params=0'
d6_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=6228792&params=0'
d7_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=2764875&params=0'
d8_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=6888445&params=0'
d10_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=6228121&params=0'
d11_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=6846181&params=0'
binhthanh_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=3797166&params=0'
phunhuan_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=3851694&params=0'
tanbinh_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=6846177&params=0'
tanphu_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=6846128&params=0'
binhtan_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=6909710&params=0'
binhchanh_geo_url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=7157268&params=0'

d1_geo = requests.get(d1_geo_url).json()
d2_geo = requests.get(d2_geo_url).json()
d3_geo = requests.get(d3_geo_url).json()
d4_geo = requests.get(d4_geo_url).json()
d5_geo = requests.get(d5_geo_url).json()
d6_geo = requests.get(d6_geo_url).json()
d7_geo = requests.get(d7_geo_url).json()
d8_geo = requests.get(d8_geo_url).json()
d10_geo = requests.get(d10_geo_url).json()
d11_geo = requests.get(d11_geo_url).json()
binhthanh_geo = requests.get(binhthanh_geo_url).json()
phunhuan_geo = requests.get(phunhuan_geo_url).json()
tanbinh_geo = requests.get(tanbinh_geo_url).json()
tanphu_geo = requests.get(tanphu_geo_url).json()
binhtan_geo = requests.get(binhtan_geo_url).json()
binhchanh_geo =requests.get(binhchanh_geo_url).json()

d1_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'district 1'
    },
    'geometry': d1_geo['geometries'][0]
}
d2_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'district 2'
    },
    'geometry': d2_geo['geometries'][0]
}
d3_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'district 3'
    },
    'geometry': d3_geo['geometries'][0]
}
d4_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'district 4'
    },
    'geometry': d4_geo['geometries'][0]
}
d5_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'district 5'
    },
    'geometry': d5_geo['geometries'][0]
}
d6_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'district 6'
    },
    'geometry': d6_geo['geometries'][0]
}
d7_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'district 7'
    },
    'geometry': d7_geo['geometries'][0]
}
d8_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'district 8'
    },
    'geometry': d8_geo['geometries'][0]
}
d10_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'district 10'
    },
    'geometry': d10_geo['geometries'][0]
}
d11_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'district 11'
    },
    'geometry': d11_geo['geometries'][0]
}
binhthanh_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'binh thanh district'
    },
    'geometry': binhthanh_geo['geometries'][0]
}
phunhuan_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'phu nhuan district'
    },
    'geometry': phunhuan_geo['geometries'][0]
}
tanbinh_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'tan binh district'
    },
    'geometry': tanbinh_geo['geometries'][0]
}
tanphu_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'tan phu district'
    },
    'geometry': tanphu_geo['geometries'][0]
}
binhtan_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'binh tan district'
    },
    'geometry': binhtan_geo['geometries'][0]
}
binhchanh_geo = {
    'type': 'Feature',
    'properties': {
        'district': 'binh chanh district'
    },
    'geometry': binhchanh_geo['geometries'][0]
}

In [48]:
districts_geo = {
    'type': 'FeatureCollection',
    'features': [
        d1_geo,
        d2_geo,
        d3_geo,
        d4_geo,
        d5_geo,
        d6_geo,
        d7_geo,
        d8_geo,
        d10_geo,
        d11_geo,
        binhthanh_geo,
        phunhuan_geo,
        tanbinh_geo,
        tanphu_geo,
        binhtan_geo,
        binhchanh_geo
    ],
}

### 2.3.5. Data Visualization

#### 2.3.5.1. Manual classification Visualization

In this section, we manually classify the venues merely based on their rating regardless of other features such as locations and likes. Specifically, the venues are split into 2 big groups: rating more than 7 and the others

In [129]:
coordinates = joined_X[['latitude', 'longitude']].values.copy()
rating = joined_X['rating'].copy()

hcm_map = folium.Map(location=hcm_center, tiles='CartoDB dark_matter', zoom_start=13)

# Add city border
hcm_map.choropleth(
    geo_data=hcm_geo,
    fill_color='grey', 
)

# Add district border
hcm_map.choropleth(
    geo_data=districts_geo,
    data=districts_demo,
    columns=['district', 'pop_density'],
    key_on='feature.properties.district',
    fill_color='BuGn',
    fill_opacity=0.3,
    line_opacity=0.2,
)

# Add clusters
folium.Circle(hcm_center, tooltip='radius 6,000', radius=6000, color='white', fill=False).add_to(hcm_map)
folium.Marker(hcm_center, tooltip='City Centroid').add_to(hcm_map)
clustered_venues = folium.map.FeatureGroup()

# loop through the all venues coordinates and add each to the feature group
for idx, (lat, lng) in enumerate(coordinates):
    if rating[idx] >= 8:
        color = 'darkblue'
    elif rating[idx] >= 7.5:
        color = 'blue'
    elif rating[idx] >= 7:
        color = 'lightblue'
    else:
        color = 'grey'

    clustered_venues.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color=None,
            fill=True,
            fill_color=color,
            fill_opacity=0.8
        )
    )
# Add classified venues to map
hcm_map.add_child(clustered_venues)

hcm_map

#### 2.3.5.2. DBSCAN Visualization

In [122]:
# from folium.plugins import BeautifyIcon

coordinates = joined_X[['latitude', 'longitude']].values.copy()
labels = joined_X['db_labels'].copy()

hcm_map = folium.Map(location=hcm_center, tiles='CartoDB dark_matter', zoom_start=13)

# Add city border
hcm_map.choropleth(
    geo_data=hcm_geo,
    fill_color='grey', 
)

# Add district border
hcm_map.choropleth(
    geo_data=districts_geo,
    data=districts_demo,
    columns=['district', 'pop_density'],
    key_on='feature.properties.district',
    fill_color='BuGn',
    fill_opacity=0.3,
    line_opacity=0.2,
)

# Add clusters
clustered_venues = folium.map.FeatureGroup()

# loop through the all venues coordinates and add each to the feature group
for idx, (lat, lng) in enumerate(coordinates):
    if labels[idx] == 3:
        color = 'orange'
    else:
        color = 'grey'

    clustered_venues.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color=None,
            fill=True,
            fill_color=color,
            fill_opacity=0.8
        )
    )

# Add classified venues to map
hcm_map.add_child(clustered_venues)

hcm_map

#### 2.3.5.3. K-Means Visualization

In [128]:
coordinates = joined_X[['latitude', 'longitude']].values.copy()
labels = joined_X['kmeans_labels'].copy()
colors_set = {
    0: 'grey',
    1: 'grey',
    2: 'red',
    3: 'grey',
    4: 'red',
    5: 'grey',
    6: 'grey',
    7: 'grey',
}

hcm_map = folium.Map(location=hcm_center, tiles='CartoDB dark_matter', zoom_start=13)

# Add city border
hcm_map.choropleth(
    geo_data=hcm_geo,
    fill_color='grey', 
)

# Add district border
hcm_map.choropleth(
    geo_data=districts_geo,
    data=districts_demo,
    columns=['district', 'pop_density'],
    key_on='feature.properties.district',
    fill_color='BuGn',
    fill_opacity=0.3,
    line_opacity=0.2,
)

# Add clusters
clustered_venues = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for idx, (lat, lng) in enumerate(coordinates):
    clustered_venues.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color=None,
            fill=True,
            fill_color=colors_set[labels[idx]],
            fill_opacity=0.8
        )
    )

# add incidents to map
hcm_map.add_child(clustered_venues)

In [1]:
print(1)

1
